# Adaptive Prediction Sets with Logistic Regression Classifier

In this notebook, we implement and explain multiclass conformal prediction step by step using the Iris dataset and a Logistic Regression classifier.

The goal is to construct prediction sets that contain the true class label with a guaranteed probability (coverage), typically 
1−α=90%


## Classification

### Imports

we import load_iris in order to use the famous Iris flower dataset ( 150 samples, 3 species/classes,  numeric features per sample.

In [1]:
from sklearn.datasets import load_iris

We import LogisticRegression in order to use a multiclass logistic regression classifier, which is a linear model whose ouputs go through a softmax function to produce probabilities.

In [2]:
from sklearn.linear_model import LogisticRegression

We import train_test_split in order to split the data into subsets(training/calibration/testing) so we can simulate uncertainty quantification

In [3]:
from sklearn.model_selection import train_test_split

Numpy is used for numerical operations

In [4]:
import numpy as np

### Load the data

We load the Iris dataset. X is a matrix of shape (150,4) each row is one flower and each cloumn a feature:
- sepal length
- sepal width
- petal length
- petal width


y is a vector of shape (150,) with integer labels:
- 0 = Setosa
- 1 = Versicolor
- 2 = Virginica

In [5]:
# We use return_X_y=True in order the function to return X->data and y -> the labels instead of a full dictionary
X, y = load_iris(return_X_y=True)
# We save the class names for later use
class_names = load_iris().target_names

### Split into train , calibration and test

We're dividin the dataset into three parts:
- __Training ( 60 %)__: Used to estimate model parameters w,b
- __Calibration ( 20 %)__: Used later to measure uncertainty for conformal prediction
- __Test (20%)__: Used to check how good our calibrated predictions are
<br>The parameter random_state=42 makes the split deterministic (same shuffle each time).

In [6]:
'''We keep 60% for training ( X_train, y_train) and we send 40 % to X_temp,y_temp (temporary set for calibariotn + test) '''
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)

In [7]:
'''Now we split that 40 % into two halves 20 % calibration and 20 % test '''
X_cal,X_test, y_cal, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Define and train the model

We create a logistic Regression classifier

In [8]:
'''multi_class='miltinomial --> we use the softmax version
    max_iter = 100 --> Means the optimizer can take up to 1000 iterations'''
clf = LogisticRegression(max_iter=1000, multi_class='multinomial')
''' We sove the maximum likelihood estimation (MLE) problem. 
In order to find the parameters that best describe the relationship between the features and labels. '''
clf.fit(X_train, y_train)


/home/sred/sklearn-venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

### Predict probabilities

We compute softmax probabilities for each sample. The output shape is (n_samples, n_classes) e.g for X_test with 30 samples and 3 classes-> (30,3). Each row corresponds to one sample, adn each column corresponds to a class.

In [9]:
probs_cal = clf.predict_proba(X_cal)
probs_test = clf.predict_proba(X_test)


probs_test[:5] slices teh first 5 samples from the test set. Each row gives a probability distribution over classes

In [10]:
print("Example softmax-like probabilities (first 5):")
print(probs_test[:5])

Example softmax-like probabilities (first 5):
[[4.87202425e-03 8.10442607e-01 1.84685368e-01]
 [9.45211039e-01 5.47879865e-02 9.74927821e-07]
 [8.32285804e-03 8.02451391e-01 1.89225751e-01]
 [4.77690344e-02 9.35885971e-01 1.63449949e-02]
 [9.52470786e-01 4.75286578e-02 5.55954386e-07]]


## Adaptive Prediction Sets

We want to build prediction sets that satisfy a guaranteed coverage. That means at a 90 % confidence level (a = 0.1), the true label will be inside  the predicted set ~90% of the time

## Sort class Probabilities (Descending)

 We calculate the indices of classes sorted from most likely to least likely

In [19]:
"""Each row cal_pi[i] holds the class indices ordered fom most to least probable for sample i. """
#argsort(1) sorts along axis=1 (rows, each sample)
#[:,::-1] reverses each row-> now in descending order
cal_pi = probs_cal.argsort(1)[:,::-1] # shape (n_cal, n_classes)

Now that we know the order of classes by probability, we:
- Rearrange each row of probabilites into descending order
- Take the cumulative sum along that order

## Compute Cumulative Probability Mass

In [22]:
'''np.rake_along_axis(A, indices, axis) rearranges elements of A according to indices along the given axis
cumsum(axis=1) computes the cumulative sum row by row
The resulting matrix cal_srt has the same shape as probs_cal, each row represents cumulative probability mass as we include more classes'''

cal_srt = np.take_along_axis(probs_cal,cal_pi, axis=1).cumsum(axis=1)

### Compute APS Nonconformity Scores

A nonconformity score measures how strange a true label looks under the model. 
We seek how much total probability mass lies above the true class. <br>
To compute that, we: <br>
- Sort probabilities and take cumulative sums (done above)
- Figure out where the true label sits in that ranking
- Extract the cumulative mass at that position <br>
That value represents the total mass of all classes up to and including the true one. It is the cumulative probability mass needed to include the true label


In [25]:
"""cal_pi.argsort(axis=1) This inverse the permutation from before. 
cal_pi told us, for each sample, how t go from original order --> sorted order
cal_pi.argsort() gives the reverse mapping, how to go sorted --> original
No each row tells us where each original calss index appears in the sorted ranking
--------------------
np.take_along_axis(cal_srt, cal_pi.argsort(axis=1),axis=1)
This reorders cal_srt back to the original class order (unsorted)
So now each elemnt [i, j] in the result corresponds to the cumulative probability mass (in sorted order) up to the class j
-------------------------
[np.arange(n), y_cal] We use NumPy indexing to exrtact, for each calibration sample
"""
cal_scores = np.take_along_axis(cal_srt, cal_pi.argsort(axis=1), axis=1)[np.arange(n),y_cal]

In [13]:
print(f"The shape of scores is {scores.shape}")
print("The unoncorfmoty scores for every sample are:")
print()

print(scores)


The shape of scores is (30,)
The unoncorfmoty scores for every sample are:

[0.08592586 0.03196779 0.0283417  0.05919274 0.04015562 0.11640978
 0.03032487 0.28054442 0.04013688 0.05407437 0.03808691 0.06561558
 0.38918292 0.00332068 0.14816095 0.02025326 0.29554113 0.06014
 0.01178579 0.23401479 0.04778678 0.19090126 0.08230812 0.04466245
 0.1550098  0.24934251 0.02699415 0.04166696 0.01092766 0.02493768]


### Quantile Computation

If we want 90 % coverage (α = 0.1). We compute quantile that captures 90 % lowest scores in calibration.<br> This means 90 % of calibration samples have smaller score tha the quantile

In [14]:
alpha= 0.1
n = len(scores)
# Theoretical index, it tell you which score in the sorted list of nonconformity scores to pick
k = np.ceil((n+1)*(1-alpha)) 
""" This represents the fraction of the distribtution below of the desired quantile.
 Dividing by n converts from example the theoritical index 28th of 30--->28/30=0.933.
  which NumPy understands as the 93.33rd percentile"""
q_level = k/n
qhat = np.quantile(scores, q_level, interpolation='higher') #If the quantile index falls between two calibration scores,choose the higher one — not interpolate between them.

print("Sorted scores:", np.sort(scores))
print("Quantile level:", q_level)
#q̂ is the threshold nonconformity score such that approximately (1 − α) = 90% of the calibration scores are smaller than or equal to it
print("q̂ =", qhat)

Sorted scores: [0.00332068 0.01092766 0.01178579 0.02025326 0.02493768 0.02699415
 0.0283417  0.03032487 0.03196779 0.03808691 0.04013688 0.04015562
 0.04166696 0.04466245 0.04778678 0.05407437 0.05919274 0.06014
 0.06561558 0.08230812 0.08592586 0.11640978 0.14816095 0.1550098
 0.19090126 0.23401479 0.24934251 0.28054442 0.29554113 0.38918292]
Quantile level: 0.9333333333333333
q̂ = 0.29554113157591066


We have qhat = 0.29554113 and this means that 90 % of the calibration scores s_i= 1- ptrue,i are below 0.2955 <br>
SO ur nonconformity score threshold is qhat=0.2955 <br>
No we will use this to decide, for each test sample and each class, which classes are "plausible" enough to be included in the prediction set

In [15]:
"""1-qhat--> Is the minimum acceptable probability for inclusion in the prediction set """
for i, probs in enumerate(probs_test):
    #np.where(probs >= (1-qhat)) return the indices of all classes whose predicted probability is greater than 1-q
    selected_classes = np.where(probs >= (1 - qhat))[0] #The [0] at the end extracts the array of indices
    selected_labels = class_names[selected_classes]
    print(f"Test sample {i}: Prediction set -> {selected_labels}")


Test sample 0: Prediction set -> ['versicolor']
Test sample 1: Prediction set -> ['setosa']
Test sample 2: Prediction set -> ['versicolor']
Test sample 3: Prediction set -> ['versicolor']
Test sample 4: Prediction set -> ['setosa']
Test sample 5: Prediction set -> ['virginica']
Test sample 6: Prediction set -> ['setosa']
Test sample 7: Prediction set -> ['versicolor']
Test sample 8: Prediction set -> ['setosa']
Test sample 9: Prediction set -> ['setosa']
Test sample 10: Prediction set -> []
Test sample 11: Prediction set -> ['versicolor']
Test sample 12: Prediction set -> ['setosa']
Test sample 13: Prediction set -> ['versicolor']
Test sample 14: Prediction set -> ['versicolor']
Test sample 15: Prediction set -> ['versicolor']
Test sample 16: Prediction set -> ['versicolor']
Test sample 17: Prediction set -> ['versicolor']
Test sample 18: Prediction set -> ['virginica']
Test sample 19: Prediction set -> ['versicolor']
Test sample 20: Prediction set -> ['setosa']
Test sample 21: Predict

## Empirical Coverage

In [16]:
# We construct a matrix that every row is a boolean mask and contain the True value in the indices that the probabilities are >= 1-q
prediction_sets = probs_test >= (1-qhat) 
# We check whether the true class y_test[i] is included (True) or excluded (False) in the prediction set of test sample i.
## Taking the mean of a boolean array in NumPy automatically converts: True -> 1 and False->0
empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),y_test].mean() 


print(f"The empirical coverage is: {empirical_coverage}")

The empirical coverage is: 0.9333333333333333


In 93% of out test samples,
the true label was included in the conformal prediction set. And that’s very close to your desired 
1−α=0.9
